The goal with all this import is to reduce the memory taken, I tried to optimize that cause I encoured many memory issue during my work

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import os
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import timm
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
from torch import cuda, device, autograd, tensor, no_grad, save
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


cuda.empty_cache()

Here is the begining data-preprocessing part. These variables store the paths to the main data folders (train and val) and the output text files (train.txt, val.txt, and test.txt).

In [ ]:

# Path to the folder where the images are stored
data_folder_train = './project_data/train'
data_folder_validation = './project_data/val'

# Path to the annotation file
train_file = './train.txt'
val_file = "./val.txt"
test_file = "./test.txt"


Note: The code could be further enhanced by using dictionaries to map labels to integer values and by handling errors more explicitly.


In [ ]:


def file_exist(path) :
    """This function checks if a file exists at a given path. If it does, it is removed, and a success message is printed; otherwise,
     a message about the non-existence of the file is printed. It was usefull during the project developpement when 
     I tried to see what can be the best syntax for the label."""

    if os.path.exists(path):
        os.remove(path)
        print(f"Le fichier {path} a été supprimé avec succès.")
    else:
        print(f"Le fichier {path} n'existe pas.")


def get_labels_from_path(image_path, base_folder):
    """This function takes an image path and the base folder as input and extracts labels from the directory structure.
    
    The label are in the following format : 0 for real and 1 for fake, then the type of fake is added with 2,3 or 4 corresponding respectively 
    to the type of fake manipulation : Identity, Expression and Head Pose
    """

    #It first converts the image path to a relative path with respect to the base folder.
    relative_path = os.path.relpath(image_path, start=base_folder)

    #Then, it splits the relative path and extracts the first element as the label.
    path_elements = os.path.dirname(relative_path).split(os.path.sep)
    label = path_elements[0]

    #The function returns a label based on certain conditions. Labels are returned as strings representing integers separated by commas.
    #If the label matches certain conditions (e.g., "FakeManipulation-1" or "Real-2"), specific combinations of integers are returned.
    if label == "FakeManipulation-1" or label == "FakeManipulation-2" :
        return "1,2"
    elif label == "FakeManipulation-3" or label == "FakeManipulation-4" :
        return "1,2,3"
    elif label == "FakeManipulation-5" :
        return "1,3,4"
    elif label == "Real-1" or label == "Real-2" or label == "Real-3" or label == "Real-4" :
        return "0"
    else :
        print("error")


def create_txt(data_folder, train) :
    """This function takes two parameters: data_folder (the path to the data folder) and train (a boolean indicating whether it's for the training set or not).
    It iterates over the directories and files in the given data_folder. If the file is an image, it gets the labels from the file path or any other appropriate method.
    Then, it writes the image path and labels to the text file.
    """
    if (train) :
        output_txt_file = train_file
        txt_file = open(output_txt_file, 'a')

        # Go through all the files in the data_folder
        for root, dirs, files in os.walk(data_folder):
            for file in files:

                # Vérify if the file is an image 
                if file.endswith('.jpg') :

                    # Full path of the file
                    image_path = os.path.join(root, file)

                    # Get the labels from the file path or any other appropriate method
                    labels = get_labels_from_path(image_path, data_folder)

                    # Write to the text file
                    txt_file.write(f'{image_path} {labels}\n')     
                else :
                    print("Error")
        txt_file.close()

    # If it's not for the training set, we create two files : one for the validation set and one for the test set    
    else :
        indice = 0
        output_txt_file_val = val_file
        output_txt_file_test = test_file
        txt_file_val = open(output_txt_file_val, 'a')
        txt_file_test = open(output_txt_file_test, 'a')

        # Go through all the files in the data_folder
        for root, dirs, files in os.walk(data_folder):
            for file in files:
                indice += 1
                # Vérify if the file is an image
                if file.endswith('.jpg') :
                    # Full path of the file
                    image_path = os.path.join(root, file)

                    # Get the labels from the file path or any other appropriate method
                    labels = get_labels_from_path(image_path, data_folder)
                    # Write to the text file
                    if indice % 2 == 0 :
                        txt_file_val.write(f'{image_path} {labels}\n')     
                    else :               
                        txt_file_test.write(f'{image_path} {labels}\n')     
                else :
                    print("Error")
        txt_file_val.close()
        txt_file_test.close()


In [ ]:
file_exist(train_file)
file_exist(val_file)
file_exist(test_file)

create_txt(data_folder_train, 1)
create_txt(data_folder_validation, 0)

This code defines data transformations using the torchvision library for image augmentation.     
These transformations are commonly used in training deep learning models on image data. Let me explain each part:

In [ ]:

"""transform_augmented: A sequence of image transformations for augmented data."""
transform_augmented = transforms.Compose([
    
    #Resize((224, 224)): Resizes the image to a square of size 224x224 pixels.
    transforms.Resize((224, 224)),

    #RandomRotation(degrees=15): Randomly rotates the image up to 15 degrees.
    transforms.RandomRotation(degrees=15),

    #RandomHorizontalFlip(): Randomly flips the image horizontally.
    transforms.RandomHorizontalFlip(),

    #ToTensor(): Converts the image to a PyTorch tensor.
    transforms.ToTensor(),

    #Normalize(mean=[...], std=[...]): Normalizes the pixel values of the tensor using pre-defined mean and standard deviation values.
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



This code defines a custom dataset class method __getitem__ within the MyDataset class. The method is responsible for retrieving and returning a sample from the dataset at a given index. It performs the following key steps:

Here's a breakdown of the key components:

In [ ]:
list_of_sub_labels = ["Identity", "Expression", "Head Pose"]

class MyDataset(Dataset):

    # Initializes the dataset by reading the information from the text file (txt_path).
    # Each line is split into image path and labels. The main label is extracted as an integer, and sub-labels (if present) are converted into a list of integers.
    def __init__(self, txt_path, transform=None, target_transform=None):

        fh = open(txt_path, 'r', encoding="latin-1")
        imgs = []
        for line in fh:
            line = line.rstrip()
            words = line.split(sep=' ')
            img_path = words[0]
            labels_str = words[1]

            # Séparer les labels principaux et les sous-labels (s'ils existent)
            label_parts = labels_str.split(',')
            main_label = int(label_parts[0])  # Label principal

            # Si des sous-labels sont présents, convertir en liste d'entiers
            sub_labels = list(map(int, label_parts[1:])) if len(label_parts) > 1 else []

            imgs.append((img_path, main_label, sub_labels))

        # Attributes:
        # imgs: A list of tuples, where each tuple contains the image path, main label, and a list of sub-labels.
        # transform: An optional transformation to be applied to the images.
        # target_transform: An optional transformation to be applied to the labels.
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        fh.close()



    def get_label_from_path(self, path):
    """ Reads a line from a text file specified by the path argument and extracts the label information. The line is expected to have a specific structure, such as "image_1.jpg 1,3,5," 
    where the first element is the image filename, and the second element is a comma-separated list of labels. The function returns a list of integers representing the label."""

        with open(path, 'r', encoding="latin-1") as file:
            line = file.readline().strip().split(' ')
            
            # Vérifier si la ligne contient au moins deux éléments
            if len(line) >= 2:
                label_list = list(map(int, line[1].split(',')))
                return label_list
            else:
                # Gérer le cas où la ligne ne contient pas de sous-labels
                return []


    def encode_labels(self, label_list):
        """method: Takes a list of labels (label_list) and encodes them into a binary vector of 0s and 1s. 
        The encoding assumes a fixed number of classes (5 in this case), and the resulting vector has a 1 at positions corresponding to the labels present in the input list."""

        encoded_label = [1 if i in label_list else 0 for i in range(5)]
        return encoded_label
    


    def __getitem__(self, index):
        """method:
    Retrieves the item at the specified index, which is a tuple containing the image filename (fn), main label (_), and sub-labels (sub_labels).
    Opens the image file using the PIL library, converts it to the RGB format, and extracts the label information.
    Concatenates the main label and sub-labels into a single list (label_list).
    Calls the encode_labels method to obtain the encoded representation of the label.
    Applies the specified image transformations if the transform attribute is not None.
    Returns the transformed image and the encoded label."""
        item = self.imgs[index]
        print(item)
        if len(item) == 2:
            fn, _ = item
            sub_labels = []  # No sub-labels present
        elif len(item) == 3:
            fn, _, it = item
            sub_labels = it  # Sub-labels present
        else:
            raise ValueError(f"Expected a tuple with 2 or 3 elements, got {len(item)} elements.")

        print(f"Filename: {fn}, Main Label: {_}, Sub Labels: {sub_labels}")


        img = Image.open(fn).convert('RGB')

        label_list = _ + sub_labels


        # Utiliser la fonction d'encodage pour obtenir la représentation numérique du label
        encoded_label = self.encode_labels(label_list)
        print(enc)

        if self.transform is not None:
            img = self.transform(img)

        return img, encoded_label


